In [1]:
import random
from math import inf


params = {
    "farmer_productivity": 1.5,
    "land_capacity": 120,

    "steal_rate": 3,
    "suppression_prob": 0.3,
    "confiscate_rate": 0.7,

    "tax_rate": 1.5,
    "soldier_base_pay": 2.0,

    "alpha_perception": 0.25,
    "fear_increment": 0.1,
    "fear_decay": 0.2,

    "utility_margin": 0.5,
    "switch_urge_threshold": 2.0,
}

EPS = 1e-6
FEAR_MAX = 10.0


class Person:
    """
    Agent in the farmer–bandit–soldier world.

    Each Person has:
      - a role: "farmer", "bandit", or "soldier"
      - an economic state: current wealth and last-tick flows
      - psychological state: fear, perceived violence, loyalty
      - heterogeneity: productivity, aggression, and skill multipliers
      - switching state: when and how they are allowed to change roles
    """

    def __init__(self, role, wealth=0.0, prod_mult=1.0, aggr_mult=1.0, skill_mult=1.0):
        """
        Initialize an agent with a given role and starting wealth.

        Parameters
        ----------
        role : str
            Initial role of the agent ("farmer", "bandit", "soldier").
        wealth : float, optional
            Initial wealth level (clipped to be non-negative).
        prod_mult : float, optional
            Baseline productivity multiplier (ignored in favor of randomization).
        aggr_mult : float, optional
            Baseline aggression/stealing multiplier (ignored in favor of randomization).
        skill_mult : float, optional
            Baseline combat/skill multiplier (ignored in favor of randomization).
        """
        self.role = role
        self.wealth = max(0.0, wealth)

        # Psychological state
        self.fear = 0.0
        self.perceived_violence = 0.0
        # Loyalty is slightly heterogeneous across agents
        self.loyalty = random.uniform(0.8, 1.2)

        # Role-switching state
        self.next_role = role
        self.age_in_role = 0
        # Cooldown prevents constant flipping between roles
        self.switch_cooldown = 3

        # Last-tick flows (used mainly for logging / diagnostics)
        self.last_production = 0.0
        self.last_rob_gain = 0.0
        self.last_pay = 0.0

        # Heterogeneity: each agent gets slightly different multipliers
        self.prod_mult = random.uniform(0.9, 1.1)
        self.aggr_mult = random.uniform(0.9, 1.1)
        self.skill_mult = random.uniform(0.9, 1.1)

    # --- housekeeping ---
    def reset_for_new_tick(self):
        """
        Reset per-tick state before simulation updates.

        This:
          - keeps the same current role but resets next_role
          - increments age_in_role
          - decreases switch_cooldown (if active)
          - zeroes out last-tick production/robbery/pay for fresh logging
        """
        self.next_role = self.role
        self.age_in_role += 1

        if self.switch_cooldown > 0:
            self.switch_cooldown -= 1

        self.last_production = 0.0
        self.last_rob_gain = 0.0
        self.last_pay = 0.0

    # --- role helpers ---
    def is_farmer(self):
        """Return True if this agent is currently a farmer."""
        return self.role == "farmer"

    def is_bandit(self):
        """Return True if this agent is currently a bandit."""
        return self.role == "bandit"

    def is_soldier(self):
        """Return True if this agent is currently a soldier."""
        return self.role == "soldier"

    # --- actions ---
    def produce(self, farmer_productivity, scale=1.0):
        """
        Let a farmer produce wealth in this tick.

        Effective production = prod_mult * farmer_productivity * scale.

        Parameters
        ----------
        farmer_productivity : float
            Baseline productivity per farmer.
        scale : float, optional
            Downscaling factor (e.g., crowding via land capacity).

        Returns
        -------
        float
            Amount of wealth produced this tick (0 if not a farmer).
        """
        if not self.is_farmer():
            return 0.0

        inc = max(0.0, self.prod_mult * farmer_productivity * max(0.0, scale))
        self.wealth = max(0.0, self.wealth + inc)
        self.last_production = inc
        return inc

    def attempt_rob(self, victim, steal_rate, fear_bump_robbed=0.3):
        """
        Attempt a robbery on a farmer if this agent is a bandit.

        Parameters
        ----------
        victim : Person
            Target agent, expected to be a farmer.
        steal_rate : float
            Baseline amount the bandit tries to steal.
        fear_bump_robbed : float, optional
            Increase in the victim's fear if the robbery succeeds.

        Returns
        -------
        tuple[float, bool]
            (stolen_amount, success_flag). If stolen_amount is 0, success_flag is False.
        """
        if not (self.is_bandit() and victim and victim.role == "farmer"):
            return 0.0, False

        # Bandit can steal up to aggr_mult * steal_rate, but not more than victim's wealth
        stolen = min(
            max(0.0, self.aggr_mult * steal_rate),
            max(0.0, victim.wealth),
        )
        if stolen <= 0:
            return 0.0, False

        # Transfer wealth and increase victim's fear
        victim.wealth = max(0.0, victim.wealth - stolen)
        victim.fear = min(FEAR_MAX, victim.fear + max(0.0, fear_bump_robbed))

        self.wealth = max(0.0, self.wealth + stolen)
        self.last_rob_gain = stolen
        return stolen, True

    def attempt_suppress(self, target_bandit, p_hit, confiscate_rate, fear_bump_suppressed=0.2):
        """
        Attempt to suppress a bandit if this agent is a soldier.

        Parameters
        ----------
        target_bandit : Person
            Target agent, expected to be a bandit.
        p_hit : float
            Baseline probability of a successful suppression.
        confiscate_rate : float
            Baseline amount of wealth to confiscate if the hit succeeds.
        fear_bump_suppressed : float, optional
            Increase in the bandit's fear when suppressed.

        Returns
        -------
        tuple[float, bool]
            (confiscated_amount, success_flag).
        """
        if not (self.is_soldier() and target_bandit and target_bandit.role == "bandit"):
            return 0.0, False

        # Hit probability scaled by soldier skill, clipped to [0, 1]
        hit_prob = max(0.0, min(1.0, self.skill_mult * p_hit))
        if random.random() >= hit_prob:
            return 0.0, False

        # Confiscate some of the bandit's wealth
        amt = min(
            max(0.0, self.skill_mult * confiscate_rate),
            max(0.0, target_bandit.wealth),
        )

        target_bandit.wealth = max(0.0, target_bandit.wealth - amt)
        target_bandit.fear = min(FEAR_MAX, target_bandit.fear + fear_bump_suppressed)

        self.wealth = max(0.0, self.wealth + amt)
        return amt, True

    def pay_tax(self, tax_rate):
        """
        Deduct taxes from a farmer's wealth.

        Parameters
        ----------
        tax_rate : float
            Fraction of wealth to be paid as tax.

        Returns
        -------
        float
            Amount of tax paid this tick.
        """
        if not self.is_farmer():
            return 0.0

        tax_rate = max(0.0, min(1.0, tax_rate))
        tax = tax_rate * max(0.0, self.wealth)
        self.wealth = max(0.0, self.wealth - tax)
        return tax

    def receive_pay(self, amount):
        """
        Add income (e.g., soldier wage) to this agent's wealth.

        Parameters
        ----------
        amount : float
            Amount of pay to add; ignored if non-positive.
        """
        if amount > 0:
            self.wealth = max(0.0, self.wealth + amount)
            self.last_pay = amount

    # --- psychology ---
    def update_psychology(self, global_violence, alpha, fear_inc, fear_dec):
        """
        Update perceived violence and fear based on global violence.

        Perceived violence is an exponential moving average of observed violence.
        Fear increases when perceived_violence > 0.5 and otherwise decays.

        Parameters
        ----------
        global_violence : float
            Aggregate measure of violence in the world this tick.
        alpha : float
            Smoothing factor for perceived_violence (0 = very slow, 1 = instant).
        fear_inc : float
            Increment added to fear when perceived_violence is high.
        fear_dec : float
            Amount fear decays when perceived_violence is low.
        """
        a = max(0.0, min(1.0, alpha))
        gv = max(0.0, global_violence)

        # Exponential smoothing of perceived violence
        self.perceived_violence = (1 - a) * self.perceived_violence + a * gv

        if self.perceived_violence > 0.5:
            self.fear = min(FEAR_MAX, self.fear + fear_inc)
        else:
            self.fear = max(0.0, self.fear - fear_dec)

    # --- forward-looking utility prediction ---
    def predicted_utility(self, role, world, params):
        """
        Predict this agent's utility next tick if they were in a given role.

        The prediction is based on:
          - current world population by role
          - last-tick violence (stored in world.total_violence_last_tick)
          - model parameters (productivity, tax rate, etc.)
          - this agent's own multipliers (prod_mult, aggr_mult, skill_mult)

        Parameters
        ----------
        role : str
            Hypothetical role to evaluate ("farmer", "bandit", "soldier").
        world : World
            World object providing current population counts and violence.
        params : dict
            Dictionary of global parameters (e.g., farmer_productivity, tax_rate).

        Returns
        -------
        float
            Predicted utility value for being in the given role next tick.
        """
        # Approximate next-tick populations if this agent adopted 'role'
        f = world.farmer_pop
        b = world.bandit_pop
        s = world.soldier_pop

        # Remove self from current role
        if self.is_farmer():
            f -= 1
        elif self.is_bandit():
            b -= 1
        elif self.is_soldier():
            s -= 1

        # Add self as the hypothetical role
        if role == "farmer":
            f += 1
        elif role == "bandit":
            b += 1
        elif role == "soldier":
            s += 1

        violence = world.total_violence_last_tick

        # ---- farmer ----
        if role == "farmer":
            # Simple land-capacity crowding: more farmers ⇒ lower scale
            if f > 0:
                scale = min(1.0, params["land_capacity"] / (f + EPS))
            else:
                scale = 1.0

            predicted_prod = self.prod_mult * params["farmer_productivity"] * scale
            safety_bonus = max(0.0, 0.5 - 0.02 * violence)
            return predicted_prod + safety_bonus - 0.01 * b

        # ---- bandit ----
        if role == "bandit":
            if f > 0:
                base_rob = self.aggr_mult * params["steal_rate"]
            else:
                base_rob = 0.0

            return (
                base_rob
                - 0.02 * b                     # competition among bandits
                + 0.03 * max(0, f - 10)        # more farmers ⇒ more loot
                - 0.02 * s                     # soldiers make bandit life worse
            )

        # ---- soldier ----
        if role == "soldier":
            # Rough expected tax pool from farmers next tick
            if f > 0:
                scale = min(1.0, params["land_capacity"] / (f + EPS))
                avg_prod = params["farmer_productivity"] * scale
                tax_pool = f * avg_prod * params["tax_rate"]
            else:
                tax_pool = 0.0

            base_total = params["soldier_base_pay"] * s
            if s > 0:
                pay_per = (tax_pool + base_total) / s
            else:
                pay_per = params["soldier_base_pay"]

            # Soldiers get some benefit from bandit presence but a disutility term
            return pay_per + 0.02 * b - 0.30

        # Fallback for invalid role
        return -999.0

    # --- switching with 5-tick lockout ---
    def decide_switch(self, utilities, switch_urge_threshold, utility_margin, global_tick):
        """
        Decide whether to switch roles based on fear, loyalty, and predicted utilities.

        Logic:
          - For the first few global ticks, no switching is allowed.
          - If still on cooldown, the agent keeps its current role.
          - Compute 'urge' = fear / loyalty; if urge is low, the agent stays put.
          - Otherwise, compare current role utility to the best alternative.
          - Switch only if the best alternative exceeds current utility by a margin.

        Parameters
        ----------
        utilities : dict[str, float]
            Dictionary mapping role → predicted utility for this agent (or role).
        switch_urge_threshold : float
            Minimum urge required before the agent even considers switching.
        utility_margin : float
            Minimum utility improvement required to justify switching roles.
        global_tick : int
            Current time step of the global simulation (for initial lockout).
        """
        # Early phase: freeze roles entirely
        if global_tick <= 5:
            self.next_role = self.role
            return

        # Cooldown: agent recently switched and must wait
        if self.switch_cooldown > 0:
            self.next_role = self.role
            return

        urge = self.fear / (self.loyalty + EPS)
        current_u = utilities.get(self.role, -1e9)

        # If psychological urge is too low, no switch
        if urge <= switch_urge_threshold:
            self.next_role = self.role
            return

        # Find the role with the highest utility
        best_role = self.role
        best_u = current_u
        for r, u in utilities.items():
            if u > best_u:
                best_role, best_u = r, u

        # Switch only if best role is sufficiently better
        if best_u > current_u + utility_margin:
            self.next_role = best_role
        else:
            self.next_role = self.role

    def finalize_role_switch(self):
        """
        Apply the role change decided in decide_switch().

        This:
          - updates the actual role,
          - resets age_in_role,
          - sets a short cooldown to avoid immediate flipping back.
        """
        if self.next_role != self.role:
            self.role = self.next_role
            self.age_in_role = 0
            self.switch_cooldown = 1  # small hysteresis

    # --- export for logging ---
    def to_dict(self):
        """
        Serialize the agent's state to a simple dictionary.

        Returns
        -------
        dict
            Dictionary with role, wealth, fear, perceived_violence, loyalty, and age_in_role.
        """
        return {
            "role": self.role,
            "wealth": round(self.wealth, 3),
            "fear": round(self.fear, 3),
            "perceived_violence": round(self.perceived_violence, 3),
            "loyalty": round(self.loyalty, 3),
            "age_in_role": self.age_in_role,
        }


